# TODO
 
* ## https://medium.com/analytics-vidhya/deploy-huggingface-s-bert-to-production-with-pytorch-serve-27b068026d18 

* ## https://github.com/pytorch/serve/tree/master/examples/Huggingface_Transformers

# Deploying our BERT PyTorch Model as REST EndPoint

In [9]:
!pip install -q transformers==2.8.0
!pip install -q torch==1.5.0 --upgrade --ignore-installed

In [34]:
!pip install torchserve

In [ ]:
!git clone https://github.com/pytorch/serve.git ./src_torchserve
!pip install ./src_torchserve/serve/model-archiver/

In [69]:
import boto3
import sagemaker
import pandas as pd

sess   = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client('sts').get_caller_identity().get('Account')

sm = boto3.Session().client(service_name='sagemaker', region_name=region)

# Clone the TorchServe repository and install torch-model-archiver

You'll use `torch-model-archiver` to create a model archive file (.mar). The .mar model archive file contains model checkpoints along with it’s `state_dict` (dictionary object that maps each layer to its parameter tensor).

In [1]:
!git clone https://github.com/pytorch/serve.git
!pip install serve/model-archiver/

Cloning into 'serve'...
remote: Enumerating objects: 266, done.
remote: Counting objects: 100% (266/266), done.
remote: Compressing objects: 100% (188/188), done.
remote: Total 9065 (delta 103), reused 113 (delta 32), pack-reused 8799
Receiving objects: 100% (9065/9065), 35.56 MiB | 102.30 MiB/s, done.
Resolving deltas: 100% (4853/4853), done.
Processing ./serve/model-archiver
  Using cached enum_compat-0.0.3-py3-none-any.whl (1.3 kB)
  Created wheel for torch-model-archiver: filename=torch_model_archiver-0.1.1b20200704-py3-none-any.whl size=15785 sha256=c1c0f4d303de415c8f08e76ad3c85bda60a71d6cfe68b939225c6d2d5c1b67cd
  Stored in directory: /home/ec2-user/.cache/pip/wheels/30/46/a4/0551adcd75fd74e180ed4b8c09ffca21c3f1fb910a31675843
Successfully built torch-model-archiver


# Retrieve PyTorch Models

In [6]:
%store -r s3_pytorch_model_path

In [7]:
print(s3_pytorch_model_path)

s3://sagemaker-us-east-1-806570384721/models/pytorch/pytorch_model.pt


In [10]:
%store -r s3_transformer_pytorch_model_path

In [11]:
print(s3_transformer_pytorch_model_path)

s3://sagemaker-us-east-1-806570384721/models/transformer-pytorch/


In [12]:
!aws s3 cp --recursive $s3_transformer_pytorch_model_path ./Transformer_model/

download: s3://sagemaker-us-east-1-806570384721/models/transformer-pytorch/config.json to Transformer_model/config.json
download: s3://sagemaker-us-east-1-806570384721/models/transformer-pytorch/pytorch_model.bin to Transformer_model/pytorch_model.bin


# Create TorchServe Model Archive File

Once, setup_config.json, sample_text.txt and index_to_name.json are set properly, we can go ahead and package the model and start serving it. The artifacts realted to each operation mode (such as sample_text.txt, index_to_name.json) can be place in their respective folder. 

In [ ]:
#!torch-model-archiver 
#    --model-name "bert" \
#    --version 1.0 \
#    --serialized-file ./bert_model/pytorch_model.bin \
#    --extra-files "./bert_model/config.json" \
#    --handler "./transformers_classifier_torchserve_handler.py"

In [54]:
model_name = 'DistilBertForSequenceClassification'

In [17]:
!torch-model-archiver \
    --model-name $model_name \
    --version 1.0 \
    --serialized-file Transformer_model/pytorch_model.bin \
    --handler ./src_torchserve/Transformer_handler_generalized.py \
    --extra-files "./Transformer_model/config.json,./src_torchserve/setup_config.json,./src_torchserve/Seq_classification_artifacts/index_to_name.json"

In [18]:
!ls ./*.mar

./DistilBertForSequenceClassification.mar


# Registering the Model on TorchServe and Running Inference

To register the model on TorchServe using the above model archive file, we run the following commands:

In [23]:
!mkdir ./model_store
!mv ./DistilBertForSequenceClassification.mar ./model_store/

mkdir: cannot create directory ‘./model_store’: File exists


In [40]:
!torchserve \
--start \
--model-store model_store \
--models distilbert-pytorch=DistilBertForSequenceClassification.mar

Removing orphan pid file.


## To run the inference using our registered model, open a new terminal and run: 

In [42]:
!curl -X POST http://127.0.0.1:8080/predictions/distilbert-pytorch -T ./src_torchserve/Seq_classification_artifacts/sample_text.txt

curl: (7) Failed to connect to 127.0.0.1 port 8080: Connection refused


# Prepare the Model for SageMaker Deployment

## Upload .mar to S3

In [44]:
torchserve_mar = 'DistilBertForSequenceClassification.mar'

In [45]:
s3_torchserve_mar = 's3://{}/models/torchserve/{}'.format(bucket, torchserve_mar)
print(s3_torchserve_mar)

s3://sagemaker-us-east-1-806570384721/models/torchserve/DistilBertForSequenceClassification.mar


In [46]:
!aws s3 cp ./model_store/$torchserve_mar $s3_torchserve_mar

upload: model_store/DistilBertForSequenceClassification.mar to s3://sagemaker-us-east-1-806570384721/models/torchserve/DistilBertForSequenceClassification.mar


In [47]:
%store s3_torchserve_mar

Stored 's3_torchserve_mar' (str)


In [56]:
!tar cvfz ./DistilBertForSequenceClassification.tar.gz \
    ./model_store/DistilBertForSequenceClassification.mar


./model_store/DistilBertForSequenceClassification.mar


In [62]:
s3_torchserve_tar = 's3://{}/models/torchserve/DistilBertForSequenceClassification.tar.gz'.format(bucket)

In [63]:
!aws s3 cp ./DistilBertForSequenceClassification.tar.gz $s3_torchserve_tar

upload: ./DistilBertForSequenceClassification.tar.gz to s3://sagemaker-us-east-1-806570384721/models/torchserve/DistilBertForSequenceClassification.tar.gz


In [64]:
%store s3_torchserve_tar

Stored 's3_torchserve_tar' (str)


### Create an Amazon ECR registry
Create a new docker container registry for your torchserve container images.

In [65]:
registry_name = 'torchserve'
!aws ecr create-repository --repository-name {registry_name}


An error occurred (RepositoryAlreadyExistsException) when calling the CreateRepository operation: The repository with name 'torchserve' already exists in the registry with id '806570384721'


### Build a TorchServe Docker container and push it to Amazon ECR

In [70]:
image_label = 'v1'
image = f'{account_id}.dkr.ecr.{region}.amazonaws.com/{registry_name}:{image_label}'

In [81]:
!docker build -t {registry_name}:{image_label} -f ./src_torchserve/Dockerfile ./src_torchserve
!$(aws ecr get-login --no-include-email --region {region})
!docker tag {registry_name}:{image_label} {image}
!docker push {image}

Sending build context to Docker daemon  60.51MB
Step 1/16 : FROM ubuntu:18.04
 ---> 8e4ce0a6ce69
Step 2/16 : ENV PYTHONUNBUFFERED TRUE
 ---> Using cache
 ---> c7057762ad65
Step 3/16 : RUN apt-get update &&     DEBIAN_FRONTEND=noninteractive apt-get install --no-install-recommends -y     fakeroot     ca-certificates     dpkg-dev     g++     python3-dev     openjdk-11-jdk     curl     vim     && rm -rf /var/lib/apt/lists/*     && cd /tmp     && curl -O https://bootstrap.pypa.io/get-pip.py     && python3 get-pip.py
 ---> Using cache
 ---> ddeae91e0f53
Step 4/16 : RUN update-alternatives --install /usr/bin/python python /usr/bin/python3 1
 ---> Using cache
 ---> 5d1f21b4f00d
Step 5/16 : RUN update-alternatives --install /usr/local/bin/pip pip /usr/local/bin/pip3 1
 ---> Using cache
 ---> 8f36e4816e10
Step 6/16 : RUN pip install --no-cache-dir psutil                 --no-cache-dir torch                 --no-cache-dir torchvision
 ---> Using cache
 ---> 2cfef5994910
Step 7/16 : ADD serve ser

### Deploy endpoint and make prediction using Amazon SageMaker SDK

In [84]:
import time

In [82]:
from sagemaker.model import Model
from sagemaker.predictor import RealTimePredictor

sm_model_name = 'torchserve-distilbert'

torchserve_model = Model(model_data = s3_torchserve_tar, 
                         image = image,
                         role  = role,
                         predictor_cls=RealTimePredictor,
                         name  = sm_model_name)

In [85]:
endpoint_name = 'torchserve-endpoint-' + time.strftime("%Y-%m-%d-%H-%M-%S", time.gmtime())

predictor = torchserve_model.deploy(instance_type='ml.m4.xlarge',
                                    initial_instance_count=1,
                                    endpoint_name = endpoint_name)

---------------!

# _Wait Until the ^^ Endpoint ^^ is Deployed_

## Test the TorchServe hosted model

In [91]:
#file_name = './src_torchserve/sample_text.txt'
#with open(file_name, 'rb') as f:
#    payload = f.read()
#    payload = payload
#    
#response = predictor.predict(data=payload)
#print(*json.loads(response), sep = '\n')

In [99]:
import json
    
reviews = ["This is great!", 
           "This is terrible."]

predicted_classes = predictor.predict(reviews)

for predicted_class, review in zip(predicted_classes, reviews):
    print('[Predicted Star Rating: {}]'.format(predicted_class), review)

ModelError: An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from model with message "{
  "code": 400,
  "type": "BadRequestException",
  "message": "Parameter model_name is required."
}
". See https://us-east-1.console.aws.amazon.com/cloudwatch/home?region=us-east-1#logEventViewer:group=/aws/sagemaker/Endpoints/torchserve-endpoint-2020-07-04-19-25-55 in account 806570384721 for more information.